In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!unzip -q /content/drive/MyDrive/eeg-code.zip -d /content/

In [ ]:
%cd /content/eeg-code

/content/eeg-code


In [ ]:
!ls

eeg-code			       filtered_eeg_data       requirements.txt
extract_features_augmented.py	       __MACOSX		       train_dl_models_pytorch.py
extract_features_filtered_windowed.py  preprocess_filtered.py  train_models_filtered_tuned.py


In [ ]:
!python3 preprocess_filtered.py \
--input /content/eeg-code/filtered_eeg_data \
--output /content/eeg-code/filtered_clean_data \
--fs 128


--- Preprocessing Filtered EEG Data ---
Input folder: /content/eeg-code/filtered_eeg_data
Output folder: /content/eeg-code/filtered_clean_data
Sampling Frequency: 128.0 Hz
--------------------------------------
Found gesture subfolders: ['close', 'index', 'open', 'victory']

Found 36 total files to process.
Preprocessing Filtered Files: 100% 36/36 [00:35<00:00,  1.02it/s]

Preprocessing finished in 35.46 seconds.

--- Preprocessing Summary ---
Attempted: 36 files.
Successfully saved: 36 files.
Output files found in '/content/eeg-code/filtered_clean_data': 36
SUCCESS!
-------------------------------


In [ ]:
!pip install nolds

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 21.8 MB/s eta 0:00:00


In [ ]:
!python3 extract_features_augmented.py \
--input /content/eeg-code/filtered_clean_data \
--output /content/eeg-code/extracted_features_aug \
--fs 128 \
--jobs -1 \
--augment_noise True --noise_level 0.02


--- EEG Windowed Feature Extraction (with Augmentation) ---
Input folder: /content/eeg-code/filtered_clean_data
Output base: /content/eeg-code/extracted_features_aug
Params: fs=128.0, Win=1.5s, Step=0.5s
Features/channel: 14 => Total: 196
Augment Noise: True (Level: 0.02)
Augment Scale: False (Range: 0.1)
Parallel jobs: All
--------------------------------------------------------

Found 36 cleaned files for windowed extraction.
Window: 1.5s (192 samples), Step: 0.5s (64 samples)
Augmentation: Noise=True (Level=0.02), Scale=False (Range=0.1)
Starting parallel processing...
Extract Augmented Features: 100% 36/36 [1:00:47<00:00, 101.31s/it]

Processing finished. Success: 36/36 files. Failed/Skipped: 0 files.
Total windows extracted (incl. augmented): 31596

Final labels (windows): [0 1 2 3] (Counts: [8348 7122 6486 9640])

Saved features ((31596, 196)) to: /content/eeg-code/extracted_features_aug_filtered_windowed_features_augmented.npy
Saved labels ((31596,)) to: /content/eeg-code/extra

In [ ]:
!python3 train_dl_models_pytorch.py --input ./filtered_clean_data --model_type cnn


🛑 WARNING: PyTorch DL with LOW NUMBER OF ORIGINAL TRIALS!
Performance may be poor/overfit. RESULTS ARE EXPERIMENTAL.

Using Nvidia GPU (CUDA device).
Found 36 cleaned files. Loading/Preprocessing each trial for PyTorch...
Loading PyTorch Data: 100% 36/36 [00:02<00:00, 16.57it/s]

Scaler saved to dl_pytorch_data_scaler.pkl
PyTorch Data Loading Complete. Shapes: X=(36, 256, 14), y=(36,)
Label distribution: {np.int64(0): np.int64(9), np.int64(1): np.int64(9), np.int64(2): np.int64(9), np.int64(3): np.int64(9)}

Train shapes: X=(27, 256, 14), y=(27,)
Test shapes: X=(9, 256, 14), y=(9,)
Using Train Batch Size: 4, Val Batch Size: 4
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

--- Training SimpleCNN_EEG_PyTorch on cuda ---
Epoch 1/100 [Train]: 100% 7/7 [00:01<00:00,  5.22it/s, loss=2.12, acc=11.1]
Epoch 1/100 [Val]: 100% 3/3 [00:00<00:00, 61.92i

In [ ]:
!python3 train_models_filtered_tuned.py \
--features /content/eeg-code/extracted_features_aug_filtered_windowed_features_augmented.npy \
--labels /content/eeg-code/extracted_features_aug_filtered_windowed_labels_augmented.npy \
--output /content/eeg-code/model_results_tuned_pca \
--use-pca


--- EEG Gesture Recognition Model Training (Filtered Data, Tuned) ---
Features: /content/eeg-code/extracted_features_aug_filtered_windowed_features_augmented.npy
Labels: /content/eeg-code/extracted_features_aug_filtered_windowed_labels_augmented.npy
Output: /content/eeg-code/model_results_tuned_pca
Dim Reduction: PCA
--------------------------------------------------------------------
Loaded features shape: (31596, 196), labels shape: (31596,)

--- Starting Model Training & Tuning ---
Input: X=(31596, 196), y=(31596,)
Input Labels: [0 1 2 3] (Counts: [8348 7122 6486 9640])
Found 4 valid classes.

Applying preprocessing (Imputer -> Scaler)...
Preprocessing applied.
Auto-PCA components for ~95% var: 70
Applying PCA (n=70)...
Shape after PCA: (31596, 70)
  Saved PCA loading plot to pca_top_feature_loadings.png
  Saved PCA variance plot to pca_explained_variance.png
Final feature shape for training: (31596, 70)
Saved preprocessing components.

Splitting data...
Train shape: (23697, 70), T

In [ ]:
!pip install xgboost


In [ ]:
!python3 train_models_filtered_randomized.py \
--features /content/eeg-code/extracted_features_aug_filtered_windowed_features_augmented.npy \
--labels /content/eeg-code/extracted_features_aug_filtered_windowed_labels_augmented.npy \
--output /content/eeg-code/model_results_tuned_pca \
--use-pca


--- EEG Gesture Recognition Model Training (Filtered Data, RandomizedSearchCV) ---
Features: /content/eeg-code/extracted_features_aug_filtered_windowed_features_augmented.npy
Labels: /content/eeg-code/extracted_features_aug_filtered_windowed_labels_augmented.npy
Output: /content/eeg-code/model_results_tuned_pca
Tuning Iterations (n_iter): 15
Dim Reduction: PCA
----------------------------------------------------------------------------------
Loaded features shape: (31596, 196), labels shape: (31596,)

--- Starting Model Training & Tuning (RandomizedSearchCV, n_iter=15) ---
Input: X=(31596, 196), y=(31596,)
Input Labels: [0 1 2 3] (Counts: [8348 7122 6486 9640])
Found 4 valid classes.

Applying preprocessing (Imputer -> Scaler)...
Preprocessing applied.
Auto-PCA components for ~95% var: 70
Applying PCA (n=70)...
Shape after PCA: (31596, 70)
  Saved PCA variance plot to pca_explained_variance.png
Final feature shape for training: (31596, 70)
Saved preprocessing components.

Splitting da